In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install --upgrade transformers
!pip install unsloth

In [2]:
import warnings
import torch
from kaggle_secrets import UserSecretsClient
import os

warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Your device:", device)
user_secrets = UserSecretsClient()
os.environ["WANDB_API_KEY"]=user_secrets.get_secret("WANDB_API_KEY")
os.environ["WANDB_PROJECT"]="Text2Cypher"

Your device: cuda


In [3]:
from unsloth import FastLanguageModel

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.2: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla P100-PCIE-16GB. Max memory: 15.888 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 6.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers via:
`pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"`


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

In [5]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

from datasets import load_dataset

trainset = load_dataset('json', data_files='/kaggle/input/text-to-cypher/training_set.jsonl', split="train")
validationset = load_dataset('json', data_files='/kaggle/input/text-to-cypher/validation_set.jsonl', split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
trainset = trainset.map(formatting_prompts_func, batched = True,)
validationset = validationset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/3363 [00:00<?, ? examples/s]

Map:   0%|          | 0/178 [00:00<?, ? examples/s]

We look at how the conversations are structured for item 5:

In [7]:
trainset[5]["conversations"]

[{'role': 'system',
  'content': 'You are an experienced Cypher developer and English-speaking recruiter and a helpful assistant designed to output JSON.\nConsider the Neo4j graph database with the following schema:\n\nNode properties are the following:\n    Person {{ID: STRING, name: STRING, phone: STRING, dob: DATE, summary: STRING, location: STRING, portfolio: STRING}}\n    Publication {{CATEGORY: STRING}}\n    Certification {{CATEGORY: STRING}}\n    Education {{NAME: STRING, Field: STRING}}\n    WorkPlace {{NAME: STRING, domain_expertise: STRING, size: STRING}}\n    Skills {{NAME: STRING, category: STRING}}\n    Position {{NAME: STRING, description: STRING}}\n\nRelationship properties are the following:\n    Achieve {{CONFERENCE: STRING, Date: DATE}}\n    Study_at {{DEGREE: STRING, start_year: INT, end_year: INT, grade: STRING, achievement: STRING}}\n    Worked_at {{ID: STRING, duration: INT, role: STRING, start_date: DATE, end_date: DATE, responsibilities: STRING, achievements: ST

And we see how the chat template transformed these conversations.

In [8]:
print(trainset[5]["text"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are an experienced Cypher developer and English-speaking recruiter and a helpful assistant designed to output JSON.
Consider the Neo4j graph database with the following schema:

Node properties are the following:
    Person {{ID: STRING, name: STRING, phone: STRING, dob: DATE, summary: STRING, location: STRING, portfolio: STRING}}
    Publication {{CATEGORY: STRING}}
    Certification {{CATEGORY: STRING}}
    Education {{NAME: STRING, Field: STRING}}
    WorkPlace {{NAME: STRING, domain_expertise: STRING, size: STRING}}
    Skills {{NAME: STRING, category: STRING}}
    Position {{NAME: STRING, description: STRING}}

Relationship properties are the following:
    Achieve {{CONFERENCE: STRING, Date: DATE}}
    Study_at {{DEGREE: STRING, start_year: INT, end_year: INT, grade: STRING, achievement: STRING}}
    Worked_at {{ID: STRING, duration: INT, role: STRING, 

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = trainset,
    eval_dataset = validationset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        per_device_eval_batch_size = 8,
        gradient_accumulation_steps = 4,
        eval_accumulation_steps = 4,
        warmup_steps = 10,
        num_train_epochs = 2,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 26,
        eval_steps = 26,
        eval_strategy="steps",
        logging_strategy="steps",
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "Llama3.1-8B-Instruct",
        report_to = "wandb"
    ),
)

Map (num_proc=2):   0%|          | 0/3363 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/178 [00:00<?, ? examples/s]

We also use Unsloth's train_on_responses_only method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [10]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/3363 [00:00<?, ? examples/s]

Map:   0%|          | 0/178 [00:00<?, ? examples/s]

We verify masking is actually done:

In [11]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are an experienced Cypher developer and English-speaking recruiter and a helpful assistant designed to output JSON.\nConsider the Neo4j graph database with the following schema:\n\nNode properties are the following:\n    Person {{ID: STRING, name: STRING, phone: STRING, dob: DATE, summary: STRING, location: STRING, portfolio: STRING}}\n    Publication {{CATEGORY: STRING}}\n    Certification {{CATEGORY: STRING}}\n    Education {{NAME: STRING, Field: STRING}}\n    WorkPlace {{NAME: STRING, domain_expertise: STRING, size: STRING}}\n    Skills {{NAME: STRING, category: STRING}}\n    Position {{NAME: STRING, description: STRING}}\n\nRelationship properties are the following:\n    Achieve {{CONFERENCE: STRING, Date: DATE}}\n    Study_at {{DEGREE: STRING, start_year: INT, end_year: INT, grade: STRING, achievement: STRING}}\n    Worked_at {{ID: STRING, duration

In [12]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                                                                                                                                                                                                                                                                                                                       \n\nMATCH (pos:Position {NAME: "Frontend Developer"})-[:Relevant_to]->(s:Skills)-[:Relevant_to]->(c:Certification) RETURN pos.NAME AS PositionName, c.CATEGORY AS CertificationCategory<|eot_id|>'

We can see the System and Instruction prompts are successfully masked!

In [13]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla P100-PCIE-16GB. Max memory = 15.888 GB.
5.984 GB of memory reserved.


In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,363 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 210
 "-____-"     Number of trainable parameters = 41,943,040


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers and Unsloth!


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ngocnguyen14073. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
26,0.213500,0.065711
52,0.062200,0.045981
78,0.048200,0.041046
104,0.040700,0.035606
130,0.031300,0.033628
156,0.030800,0.029052
182,0.028700,0.027668
208,0.028900,0.027038


In [15]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

23307.7642 seconds used for training.
388.46 minutes used for training.
Peak reserved memory = 12.758 GB.
Peak reserved memory for training = 6.774 GB.
Peak reserved memory % of max memory = 80.3 %.
Peak reserved memory for training % of max memory = 42.636 %.


In [16]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [19]:
!zip -r model.zip /kaggle/working

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/huggingface_tokenizers_cache/ (stored 0%)
  adding: kaggle/working/huggingface_tokenizers_cache/.locks/ (stored 0%)
  adding: kaggle/working/huggingface_tokenizers_cache/.locks/models--unsloth--meta-llama-3.1-8b-instruct-bnb-4bit/ (stored 0%)
  adding: kaggle/working/huggingface_tokenizers_cache/.locks/models--unsloth--meta-llama-3.1-8b-instruct-bnb-4bit/165b36bc2293dda9a2fb3c0daf6577d9eba9df7a.lock (stored 0%)
  adding: kaggle/working/huggingface_tokenizers_cache/.locks/models--unsloth--meta-llama-3.1-8b-instruct-bnb-4bit/d185c9e9b9d1e3f4e0c613d27efc412efdc3dde1.lock (stored 0%)
  adding: kaggle/working/huggingface_tokenizers_cache/.locks/models--unsloth--meta-llama-3.1-8b-instruct-bnb-4bit/5cc5f00a5b203e90a27a3bd60d1ec393b07971e8.lock (stored 0%)
  adding: kaggle/working/huggingface_tokenizers_cache/models--unsloth--meta-llama-3.1-8b-instruct-bnb-4bit/ (stored 0%)
  adding: kaggle/working/huggingface_tokenizers_cache/mode

In [21]:
from IPython.display import FileLink
FileLink(r'model.zip')

/kaggle/working/model.zip